Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [ ]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import heapq

In [ ]:
PUZZLE_DIM = 5
action = namedtuple('Action', ['pos1', 'pos2'])

This algorithm solves the puzzle problem using the A* algorithm with a priority queue. 
The goal is to find the shortest path (in terms of moves) to get the puzzle from an initial state to a goal state [1,2, ..., n-1, 0].

available_actions() returns a list of valid moves that can be performed from the current position of the empty square;

do_action() makes a move on the current state of the puzzle, returning a new state;

goal_positions precalculates the target positions for each value in the puzzle, to speed up the calculation of heuristics using the divmod() function;

heuristic() computes a heuristic value to estimate the remaining cost to reach the goal state via the parameters Manhattan distance and number of misplaced tiles;

solve_puzzle() solves the puzzle using the A* algorithm;


In [ ]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

goal_positions = {value: divmod(value - 1, PUZZLE_DIM) for value in range(1, PUZZLE_DIM**2)}

def heuristic(state: np.ndarray, goal: np.ndarray) -> int:
    """ Optimizing heuristics based on Manhattan distance and the number of misplaced tiles. """
    manhattan_distance = 0
    misplacement_count = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0: 
                goal_x, goal_y = divmod(value - 1, PUZZLE_DIM)
                manhattan_distance += abs(x - goal_x) + abs(y - goal_y)
                if goal[x, y] != value:
                    misplacement_count += 1
    
    return manhattan_distance + 2 * misplacement_count  

def solve_puzzle(start_state: np.ndarray, goal_state: np.ndarray):
    """Solve puzzle n²-1 using the A* algorithm with an optimized priority queue."""
    frontier = []
    heapq.heappush(frontier, (0, start_state.tolist(), [start_state.tolist()], 0))
    visited = set()

    while frontier:

        f_cost, current_state, path, g_cost = heapq.heappop(frontier)
        current_state = np.array(current_state)

        if np.array_equal(current_state, goal_state):
            return path

        state_str = current_state.tobytes()
        if state_str in visited:
            continue
        visited.add(state_str)

        for act in available_actions(current_state):
            new_state = do_action(current_state, act)
            new_g_cost = g_cost + 1
            new_f_cost = new_g_cost + heuristic(new_state, goal_state)
            new_path = path + [new_state.tolist()]
            heapq.heappush(frontier, (new_f_cost, new_state.tolist(), new_path, new_g_cost))

    return None


To ensure that the initial puzzle can be traced back to the final form of the problem (is solvable), the final solution to the problem is messed up by choosing a number of random moves, dictated by the RANDOMIZE_STEPS parameter.
The number of RANDOMIZED_STEPS indicated is 150 and allows you to have a starting solution with a good level of randomization especially with PUZZLE_DIM <= 8, for larger dimensions it is inevitably necessary to increase the number of RANDOMIZED_STEPS, to the detriment of performance in terms of time.

In [ ]:
RANDOMIZE_STEPS = 150

if PUZZLE_DIM == 0:
    print("Solution found in 0 moves!")
    print("Step 0:")
    print([])
elif PUZZLE_DIM == 1:
    state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    print("Solution found in 0 moves!")
    print("Step 0:")
    print(state)
else:
    state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    for _ in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):

        state = do_action(state, choice(available_actions(state)))

    goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

    
    solution = solve_puzzle(state, goal_state)

    if solution:
        print(f"Solution found in {len(solution) - 1} moves!")  
        for step, s in enumerate(solution):
            print(f"Step {step}:")
            print(np.array(s))  
    else:
        print("No solution found.")
